In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath("./")))
from src.models.ollama_model import OllamaModel
from src.tools.base import Tool
from src.tools import model_tools
from src.prompts.system_prompts import AGENT_SYSTEM_PROMPT

In [29]:
model_service = OllamaModel(
    model="llama3.1:latest",
    system_prompt=AGENT_SYSTEM_PROMPT.format(tool_descriptions=[tool.tool_description for tool in tools]),
    is_tool_use_model=False
)

In [4]:
# check the generate method
response = model_service.generate_text_api_call("Who is the Prime minister of India?")

[MODEL]: REQUEST RESPONSE: 200


Response from OllamaModel::llama3.1:latest=As of my last update, the Prime Minister of India is Narendra Modi. He has been serving in this position since May 26, 2014. His tenure began with his election as the leader of the Bharatiya Janata Party (BJP) and was confirmed through a decisive victory in the 2014 Lok Sabha elections.


In [2]:
def add(num1: int, num2: int) -> int:
    "adds two int"
    return num1 + num2


multiply_fn = getattr(model_tools, 'multiply')

add_tool = Tool.from_function(function=add, return_direct=False)
multiply_tool = Tool.from_function(function=multiply_fn, return_direct=False)

tools = [add_tool, multiply_tool]

In [7]:
model_service.system_prompt

'\nToday\'s Date: 14 September 2024\n# Tool Instructions\n- You will decide which tools to use, if any from the given the toolbox.\n\n\nYou will generate the following JSON response:\n\n"tool_choice": "name_of_the_tool",\n"tool_input": "inputs_to_the_tool"\n\n- `tool_choice`: The name of the tool you want to use. It must be a tool from your toolbox \n                or "no tool" if you do not need to use a tool.\n- `tool_input`: The specific inputs required for the selected tool.\n                If no tool, just provide a response to the query.\n\nHere is a list of your tools along with their descriptions:\n[\'add(num1: int, num2: int) -> int\\nadds two int\', \'multiply(a: int, b: int) -> int\\nMultiplies two integers and returns the result integer.\\n\\n    Args:\\n        a (int): The first integer.\\n        b (int): The second integer.\\n\\n    Returns:\\n        int: The product of a and b.\\n    \']\n\nReminder:\n- Function calls MUST follow the specified format\n- Required par

In [30]:
response = model_service.chat({"role": "user", "content": "What is 3789 * 1024?"}, tools=tools)


[MODEL]: {'model': 'llama3.1:latest', 'created_at': '2024-09-14T13:53:47.620931949Z', 'message': {'role': 'assistant', 'content': '{\n  "tool_choice": "multiply",\n  "tool_input": {\n    "a": 3789,\n    "b": 1024\n  }\n}'}, 'done_reason': 'stop', 'done': True, 'total_duration': 499211357, 'load_duration': 16034225, 'prompt_eval_count': 291, 'prompt_eval_duration': 44255000, 'eval_count': 35, 'eval_duration': 315285000}



In [31]:
response

{'role': 'assistant',
 'content': '{\n  "tool_choice": "multiply",\n  "tool_input": {\n    "a": 3789,\n    "b": 1024\n  }\n}'}

## Test Agent

In [27]:
from src.agents.function_calling_agent import FunctionCallingAgent

function_calling_agent = FunctionCallingAgent(
    model_name='llama3.1:latest',
    model_service="ollama",
    tools=[add_tool],
    stop_token=None,
    system_prompt=None,
    is_tool_use_model=False
)

In [28]:
function_calling_agent.chat("what is the result of 330 + 222222", max_retries=3)


[MODEL]: {'model': 'llama3.1:latest', 'created_at': '2024-09-14T13:53:24.645331679Z', 'message': {'role': 'assistant', 'content': '{\n"tool_choice": "calculator",\n"tool_input": {\n"value1": "330",\n"value2": "222222"\n}\n}'}, 'done_reason': 'stop', 'done': True, 'total_duration': 409797013, 'load_duration': 13011469, 'prompt_eval_count': 210, 'prompt_eval_duration': 14812000, 'eval_count': 29, 'eval_duration': 257254000}


[MODEL]: {'model': 'llama3.1:latest', 'created_at': '2024-09-14T13:53:25.439579675Z', 'message': {'role': 'assistant', 'content': '{ "tool_choice": "no tool", \n"tool_input": "The result of 330 + 222222 is 222552." } \n\n \n           \n            \n\n\n               \n   \n\n\n \n       \n\n\n   \n\n\n\n    \n   \n\n\n\n         \n\n \n \n \n\n             '}, 'done_reason': 'stop', 'done': True, 'total_duration': 766767325, 'load_duration': 9262345, 'prompt_eval_count': 222, 'prompt_eval_duration': 20354000, 'eval_count': 54, 'eval_duration': 613909000}



'{ "tool_choice": "no tool", \n"tool_input": "The result of 330 + 222222 is 222552." } \n\n \n           \n            \n\n\n               \n   \n\n\n \n       \n\n\n   \n\n\n\n    \n   \n\n\n\n         \n\n \n \n \n\n             '

In [4]:
from src.tools import model_tools
import importlib

print(model_tools.__all__)

['multiply', 'add', 'subtract', 'divide']


In [24]:
from pydantic import BaseModel

In [28]:
add_fn = getattr(model_tools, 'add')

### Truncation logic

In [5]:
from src.utils import truncate_chat_history

chat_history = [{"role": "user", "content": "Say my name"}, {"role": "assistant", "content": "Heisenberg"}] * 1000

chat_history.append({"role": "assistant", "content": "I'm the one who knocks !"})
len(chat_history)

2001

In [6]:
final_messages, final_token_count = truncate_chat_history(chat_history, token_limit=1536)

Total token count: 4006
Reduced token count: 4004
Reduced token count: 4002
Reduced token count: 4000
Reduced token count: 3998
Reduced token count: 3996
Reduced token count: 3994
Reduced token count: 3992
Reduced token count: 3990
Reduced token count: 3988
Reduced token count: 3986
Reduced token count: 3984
Reduced token count: 3982
Reduced token count: 3980
Reduced token count: 3978
Reduced token count: 3976
Reduced token count: 3974
Reduced token count: 3972
Reduced token count: 3970
Reduced token count: 3968
Reduced token count: 3966
Reduced token count: 3964
Reduced token count: 3962
Reduced token count: 3960
Reduced token count: 3958
Reduced token count: 3956
Reduced token count: 3954
Reduced token count: 3952
Reduced token count: 3950
Reduced token count: 3948
Reduced token count: 3946
Reduced token count: 3944
Reduced token count: 3942
Reduced token count: 3940
Reduced token count: 3938
Reduced token count: 3936
Reduced token count: 3934
Reduced token count: 3932
Reduced token 

In [9]:
len(final_messages)

# look at the last messages to confirm if the earlier messages have been truncated
final_messages[-3:]

[{'role': 'user', 'content': 'Say my name'},
 {'role': 'assistant', 'content': 'Heisenberg'},
 {'role': 'assistant', 'content': "I'm the one who knocks !"}]

In [2]:
import datetime as dt
dt.datetime.now().strftime("%d %B %Y")

'14 September 2024'